In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression


def calculate_mutual_information(data, target_column):
    """
    Calcula a informação mútua entre cada variável e a variável alvo.

    Args:
        data (pd.DataFrame): Dataset contendo as variáveis.
        target_column (str): Nome da coluna alvo.

    Returns:
        pd.Series: Informação mútua de cada variável em relação à variável alvo.
    """
    features = data.drop(columns=[target_column])
    target = data[target_column]

    # Calcula a informação mútua
    mi_scores = mutual_info_regression(features, target, random_state=42)
    return pd.Series(mi_scores, index=features.columns)


def generate_new_features(data, target_column, num_new_features):
    """
    Gera novas variáveis combinando as variáveis com maior informação mútua.

    Args:
        data (pd.DataFrame): Dataset original.
        target_column (str): Nome da coluna alvo.
        num_new_features (int): Número de novas variáveis a serem geradas.

    Returns:
        pd.DataFrame: Dataset com as novas variáveis adicionadas.
    """
    mi_scores = calculate_mutual_information(data, target_column)
    top_features = mi_scores.nlargest(min(num_new_features, len(mi_scores))).index

    new_features_generated = 0
    for i in range(len(top_features)):
        for j in range(i + 1, len(top_features)):
            if new_features_generated >= num_new_features:
                break

            new_feature_name = f"MI_{top_features[i]}_{top_features[j]}"
            data[new_feature_name] = data[top_features[i]] * data[top_features[j]]
            new_features_generated += 1

    return data


def process_datasets(input_dir, output_dir, new_feature_percentage=0.50):
    """
    Processa os datasets de um diretório, gera novas variáveis e salva os resultados em outro diretório.

    Args:
        input_dir (str): Caminho do diretório de entrada.
        output_dir (str): Caminho do diretório de saída.
        new_feature_percentage (float): Percentual de novas variáveis a serem geradas.
    """
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            print(f"Processando arquivo: {input_path}")
            try:
                # Carrega o dataset
                data = pd.read_csv(input_path, delimiter=",", skipinitialspace=True)

                # Define a coluna alvo como a última coluna do dataset
                target_column = data.columns[-1]

                # Calcula o número de novas variáveis a serem geradas
                num_features = len(data.columns) - 1
                num_new_features = max(1, int(num_features * new_feature_percentage))

                print(f"Gerando {num_new_features} novas variáveis para o arquivo {filename}...")

                # Gera novas variáveis
                data_with_new_features = generate_new_features(data, target_column, num_new_features)

                # Reorganizar as colunas para manter a coluna alvo no final
                columns = [col for col in data_with_new_features.columns if col != target_column] + [target_column]
                data_with_new_features = data_with_new_features[columns]

                # Salva o dataset transformado
                data_with_new_features.to_csv(output_path, sep=",", index=False)
                print(f"Arquivo processado e salvo em: {output_path}")

            except Exception as e:
                print(f"Erro ao processar {filename}: {e}")


# Diretórios de entrada e saída
input_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\0-saida"
output_directory = r"C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\9-saida"

# Processar arquivos
process_datasets(input_directory, output_directory, new_feature_percentage=0.50)


Processando arquivo: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\0-saida\cocomo81.txt
Gerando 8 novas variáveis para o arquivo cocomo81.txt...
Arquivo processado e salvo em: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\9-saida\cocomo81.txt
Processando arquivo: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\0-saida\encoded_china.txt
Gerando 10 novas variáveis para o arquivo encoded_china.txt...
Arquivo processado e salvo em: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\9-saida\encoded_china.txt
Processando arquivo: C:\Users\CALEO\OneDrive - Hexagon\Documents\GitHub\Software_effort_estimation\proposal\algorithms\abordagens\output\0-saida\encoded_desharnais.txt
Gerando 11 novas variáveis para o arquivo encoded_